In [1]:
import os
import json
from pathlib import Path
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from llama_index.readers.json import JSONReader
from llama_index.core.node_parser import JSONNodeParser
from llama_index.readers.file import FlatReader

In [2]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

db_name = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "lavo" #<-------------------------------------------- change this to your username !!!

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="json_data",
    embed_dim=1024,  # openai embedding dimension
)

In [4]:
# parser = JSONNodeParser()     # if we want to split the documents into nodes
llm = Ollama(model="llama3.2", request_timeout=180.0) 

# retrieving from existing db (if we already have the embeddings)

In [5]:
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="json_data",
    embed_dim=1024,  # openai embedding dimension
)

In [6]:
from llama_index.core.schema import NodeWithScore
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [7]:
embed_model = resolve_embed_model("local:BAAI/bge-m3")

In [8]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=14
)

# VectorStoreQuery is defined using this query embedding

In [12]:
query_str = "General Summarized Overview Large Capacity Cutting Machine 2?"

query_embedding = embed_model.get_query_embedding(query_str)

In [13]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [14]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [15]:
query_str = "Hello"

## to run this i need a VectorStoreQuery.. we need to explain what is it

In [16]:
response = query_engine.query(query_str)

In [17]:
print(response)

I'm happy to chat with you. However, it seems like we just started our conversation, and I don't have any specific topic or question to discuss yet. Would you like to talk about something in particular or ask a question? I'm here to help!
